The Florida DEP has run the Coastal Construction Control Line (CCCL) program since 1980.  In that time period, thousands of permits have been generated and assigned general spatial locations.  

Our purpose here is to analyze this permit history so as to be able to estimate future likely permit volumes and characteristics.

The main permit history has been extracted from internal agency systems and formated as a GIS point file.   The agency systems to date have only tracked generalized permit location, and multiple permits are shown occuring on the same exact geographic location.

In [6]:
# set up required libraries for this analysis
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
import os
import numpy as np

In [7]:
#show available datasets
!ls fbhcp_public_data

ls: fbhcp_public_data: No such file or directory


In [8]:
#unzip the historic permits file
!unzip -d fbhcp_public_data fbhcp_public_data/historic_cccl_permit_points_with_actcodes.zip

unzip:  cannot find or open fbhcp_public_data/historic_cccl_permit_points_with_actcodes.zip, fbhcp_public_data/historic_cccl_permit_points_with_actcodes.zip.zip or fbhcp_public_data/historic_cccl_permit_points_with_actcodes.zip.ZIP.


In [9]:
# read the historic permit points into a geopandas data frame
permits = gpd.read_file(os.path.join('fbhcp_public_data', "historic_cccl_permits_with_actcodes.shp"))

IOError: no such file or directory: 'fbhcp_public_data/historic_cccl_permits_with_actcodes.shp'

In [89]:
permits = permits.drop('OBJECTID', 1)
permits = permits.drop('OBJECTID_1', 1)
permits = permits.drop('PNT_PROX', 1)
permits = permits.drop('ISS_DATE', 1)
permits = permits.drop('PRMT_NUM', 1)
permits = permits.drop('LATITUDE', 1)
permits = permits.drop('LONGITUDE', 1)
permits = permits.drop('NORTH', 1)
permits = permits.drop('CodeDesc', 1)

In [90]:
# drop irrelevant columns
permits = permits.drop('ACCURACY_D', 1)
permits = permits.drop('ACCURACY_I', 1)
permits = permits.drop('CONST_ST', 1)
permits = permits.drop('DATUM', 1)
permits = permits.drop('DATUM_DESC', 1)
permits = permits.drop('DESCRIPT_1', 1)
permits = permits.drop('DES_CODE', 1)
permits = permits.drop('DesCodeInt', 1)
permits = permits.drop('EXP_DATE', 1)
permits = permits.drop('EAST', 1)
permits = permits.drop('FILE_ID', 1)
permits = permits.drop('PROXIMITY_', 1)
permits = permits.drop('SP_ZONE', 1)
permits = permits.drop('SRC', 1)
permits = permits.drop('STATUS', 1)
permits = permits.drop('VER_STAT', 1)
permits = permits.drop('Cnt_DESCRI', 1)
permits = permits.drop('ActGrpYr5', 1)
permits = permits.drop('ActCode', 1)
permits.head()

,ActCode,ActGroup,ActGrpYr5,COUNTY,Cnt_DESCRI,DESCRIPT,FRST_LOC,ISSUE_YEAR,OWN_LAST,PRMT_TYP,RNG_FROM,RNG_TO,geometry,year5
0,None,LSPs,LSPs1,BA,2301,"Decks not attached, Gazebos, Picnic Shelters, ...","16323 W Hwy 98, Panama City Beach",1983,"Catchings, Frank and Edith",CCCL,BA R-039,None,POINT (219269.6940000001 691355.1060000001),1
1,None,LSPs,LSPs1,BA,1642,Paved Area 144 Sq Ft Or Less,"16323 W Hwy 98, Panama City Beach",1983,"Catchings, Frank and Edith",CCCL,BA R-039,None,POINT (219269.6940000001 691355.1060000001),1
2,None,LSPs,LSPs1,BA,1775,"Walls - Privacy, Safety, Security and Ornamental","16323 W Hwy 98, Panama City Beach",1983,"Catchings, Frank and Edith",CCCL,BA R-039,None,POINT (219269.6940000001 691355.1060000001),1
3,None,LSPs,LSPs1,BA,2301,"Decks not attached, Gazebos, Picnic Shelters, ...","16321 W Hwy 98, Panama City Beach",1983,"Scott, William H. & Eleanor",CCCL,BA R-039,None,POINT (219284.1810000001 691346.5580000002),1
4,None,LSPs,LSPs1,BA,1775,"Walls - Privacy, Safety, Security and Ornamental","16321 W Hwy 98, Panama City Beach",1983,"Scott, William H. & Eleanor",CCCL,BA R-039,None,POINT (219284.1810000001 691346.5580000002),1


In [87]:
permits.describe()

,Cnt_DESCRI,ISSUE_YEAR,LATITUDE,LONGITUDE,NORTH,year5
count,68691.000000,68691.000000,68691.000000,68691.000000,6.869100e+04,68691.000000
mean,4695.132157,1998.122680,28.179796,82.446932,9.659887e+05,4.252857
std,3683.323019,9.346091,1.614789,2.223034,5.347431e+05,1.719207
min,1.000000,982.000000,24.547964,80.031947,7.873746e+04,0.000000
25%,1775.000000,1992.000000,26.761306,80.507244,5.090371e+05,3.000000
50%,3464.000000,1998.000000,27.716995,81.841845,8.764715e+05,4.000000
75%,7100.000000,2006.000000,29.910968,84.364131,1.200000e+06,6.000000
max,13008.000000,2012.000000,30.703781,87.522642,2.300000e+06,7.000000


In [47]:
# activity groups
permits.ActGroup.unique()

array([u'LSPs', u'RAKE', u'ARMR', u'FILL', u'LSPl', u'SFRm', u'MFRm',
       u'MFRl', None, u'DUNE', u'SFRl', u'SFRs', u'MFRs', u'LITE', u'XOVR'], dtype=object)

In [48]:
# clean up to remove some obvious data errors
permits = permits.loc[permits['ISSUE_YEAR'] >= 1980]

#show permits by specific year and activity group
permits.groupby(['ISSUE_YEAR', 'ActGroup']).size().unstack(fill_value=0)

ActGroup,ARMR,DUNE,FILL,LITE,LSPl,LSPs,MFRl,MFRm,MFRs,RAKE,SFRl,SFRm,SFRs,XOVR
ISSUE_YEAR,,,,,,,,,,,,,,
1980,68,34,61,0,68,251,7,14,0,7,0,26,0,0
1981,126,51,165,0,102,380,6,41,0,10,0,38,0,0
1982,255,51,210,0,126,475,6,33,0,9,0,51,1,0
1983,145,40,168,0,131,420,5,38,0,9,1,46,2,0
1984,207,48,323,0,134,588,15,23,0,13,2,62,4,0
1985,701,70,565,0,115,625,10,17,0,6,0,76,2,0
1986,264,29,173,0,112,336,15,9,0,3,1,83,6,0
1987,84,19,69,0,157,254,10,13,0,0,3,79,6,0
1988,73,34,180,0,217,304,3,7,0,0,0,80,7,0


In [49]:
# summarize over five-year ranges, since that's what we'll simulate in HCP
# year5
#show permits by specific year and activity group
permits.groupby(['year5', 'ActGroup']).size().unstack(fill_value=0)

ActGroup,ARMR,DUNE,FILL,LITE,LSPl,LSPs,MFRl,MFRm,MFRs,RAKE,SFRl,SFRm,SFRs,XOVR
year5,,,,,,,,,,,,,,
1,801,224,927,0,561,2114,39,149,0,48,3,223,7,0
2,1190,199,1242,0,871,1982,47,61,0,11,6,444,27,0
3,517,228,2308,0,2680,4275,62,159,0,34,7,1093,79,0
4,570,307,2045,0,3198,4701,107,253,2,2,7,1552,56,0
5,358,295,1555,4,2662,3945,108,339,28,32,11,1387,48,0
6,535,644,2936,723,2037,5376,51,280,30,32,49,874,103,0
7,272,259,1383,489,825,2051,42,22,27,22,13,341,27,24


In [50]:
#by general beaches region
# coded as SP_ZONE
permits.groupby(['SP_ZONE', 'ActGroup']).size().unstack(fill_value=0)

ActGroup,ARMR,DUNE,FILL,LITE,LSPl,LSPs,MFRl,MFRm,MFRs,RAKE,SFRl,SFRm,SFRs,XOVR
SP_ZONE,,,,,,,,,,,,,,
EAST,1856,1035,6399,671,6719,12895,285,638,50,71,60,2497,231,5
NORTH,772,768,3042,266,2972,6146,101,412,17,73,11,1950,39,16
WEST,1615,353,2955,279,3143,5403,70,213,20,37,25,1467,77,3


In [92]:
# by County
by_county = permits.groupby(['COUNTY', 'ActGroup']).size().unstack(fill_value=0)
by_county

ActGroup,ARMR,DUNE,FILL,LITE,LSPl,LSPs,MFRl,MFRm,MFRs,RAKE,SFRl,SFRm,SFRs,XOVR
COUNTY,,,,,,,,,,,,,,
BA,176,91,469,15,597,865,54,155,4,11,3,242,7,3
BE,295,88,900,56,792,1343,23,114,2,3,1,292,11,0
BO,69,48,392,46,484,1148,74,67,5,6,5,120,39,0
CH,117,27,372,21,300,590,4,43,0,1,3,278,5,0
CI,0,0,0,0,0,0,0,0,0,0,0,0,1,0
CO,154,110,670,86,829,1582,29,53,2,8,9,296,44,0
DA,105,57,570,34,601,1617,60,160,13,1,8,64,55,1
DU,55,25,199,35,352,498,12,30,2,0,4,120,5,1
ES,90,168,368,37,506,1053,9,117,7,11,4,220,4,0


In [52]:
# by Rzone
by_rzone = permits.groupby(['RNG_FROM', 'ActGroup']).size().unstack(fill_value=0)
by_rzone

ActGroup,ARMR,DUNE,FILL,LITE,LSPl,LSPs,MFRl,MFRm,MFRs,RAKE,SFRl,SFRm,SFRs,XOVR
RNG_FROM,,,,,,,,,,,,,,
BA R-001,6,9,7,0,0,6,0,0,0,2,0,0,0,0
BA R-002,2,0,9,0,6,8,0,1,0,0,0,4,0,0
BA R-002T,0,0,0,0,2,1,0,0,0,0,0,0,0,0
BA R-003,4,0,12,0,2,14,0,0,0,0,0,7,0,0
BA R-004,1,2,8,0,2,14,0,0,0,0,0,5,0,0
BA R-005,2,4,11,1,3,16,0,0,0,0,0,12,0,0
BA R-005T,0,0,0,0,2,2,0,0,0,0,0,1,0,0
BA R-006,2,1,2,0,2,6,0,1,0,1,0,0,0,0
BA R-007,1,0,1,0,1,5,1,1,0,0,0,0,0,0


In [112]:
from bokeh.plotting import figure, output_file, output_notebook, show

In [111]:
# output to static HTML file
#output_file("lines.html")
output_notebook()

x = [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
y = [i**2 for i in x]
# create a new plot with a title and axis labels
p = figure(title="CCCL Permits", x_axis_label='Year', y_axis_label='County')

p.circle(x, y, legend="Temp.")

# show the results
show(p)

Loading BokehJS ...

In [5]:
permits.first()

NameError: name 'permits' is not defined

In [4]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)
center = [34.6252978589571, -77.34580993652344]
zoom = 10
m = Map(center=center, zoom=zoom)
m

In [2]:
from ipyleaflet import Map

Map(center=[34.6252978589571, -77.34580993652344], zoom=10)

SyntaxError: invalid syntax (<ipython-input-1-1c35f686d804>, line 1)